<a href="https://colab.research.google.com/github/ris27hav/ACA-Wikipedia-Simplifier/blob/main/Final%20model/Team_4/Wikipedia_Simplifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wikipedia Simplifier

### Import pretrained text-simplification model: *MUSS*

In [ ]:
!git clone https://github.com/ris27hav/muss/

Cloning into 'muss'...
remote: Enumerating objects: 325, done.
remote: Counting objects: 100% (325/325), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 325 (delta 171), reused 274 (delta 135), pack-reused 0
Receiving objects: 100% (325/325), 4.53 MiB | 10.02 MiB/s, done.
Resolving deltas: 100% (171/171), done.


In [ ]:
!pip install -e muss/

Obtaining file:///content/muss
     |████████████████████████████████| 1.5MB 8.3MB/s 
     |████████████████████████████████| 81kB 11.9MB/s 
     |████████████████████████████████| 174kB 52.9MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
     |████████████████████████████████| 1.7MB 36.5MB/s 
     |████████████████████████████████| 28.4MB 105kB/s 
     |████████████████████████████████| 1.2MB 50.2MB/s 
     |████████████████████████████████| 61kB 10.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 89.7MB 94kB/s 
     |████████████████████████████████| 901kB 45.2MB/s 
     |████████████████████████████████| 389kB 48.0MB/s 
     |████████████████████████████████| 3.3MB 41.7MB/s 
  Cloning git://github.com/feralvam/easse.git to /tmp/pip-install-4hv7io5r/easse
  Running command git clone -q git://github.com/feralvam/easse.git /tmp/pip-install-4hv7io5r/eas

In [ ]:
%cd muss/

/content/muss


In [ ]:
%run scripts/simplify.py scripts/examples.en --model-name muss_en_wikilarge_mined

Downloading...
... 100% - 3835 MB - 27.72 MB/s - 138s
Extracting...
Downloading...
... 100% - 1 MB - 7.52 MB/s - 0s
Downloading...
... 100% - 0 MB - 4.31 MB/s - 0s
Downloading...
... 100% - 0 MB - 4.66 MB/s - 0s


INFO:fairseq_cli.generate:Namespace(all_gather_list_size=16384, batch_size=None, batch_size_valid=None, beam=5, bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', constraints=None, cpu=False, criterion='cross_entropy', curriculum=0, data='/tmp/tmp1sc0qjne', data_buffer_size=10, dataset_impl='raw', ddp_backend='c10d', decoding_format=None, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_port=-1, distributed_rank=0, distributed_world_size=1, distributed_wrapper='DDP', diverse_beam_groups=-1, diverse_beam_strength=0.5, diversity_rate=-1.0, empty_cache_freq=0, eval_bleu=False, eval_bleu_args=None, eval_bleu_detok='space', eval_bleu_detok_args=None, eval_bleu_print_samples=False, eval_bleu_remove_bpe=None, eval_tokenized_bleu=False, fast_stat_sync=False, find_unused_parameters=False, fix_batches_to_gpus=False, fixed_validation_seed=None, forc

---
### Get desired wikipedia article

#####**Extract wikipedia articles**

In [ ]:
%pip install wikipedia

In [ ]:
# use text-wrapping in output
from IPython.display import HTML, display

def set_css():
    display(HTML('''<style>pre {white-space: pre-wrap;}</style>'''))

get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
import wikipedia
from collections import OrderedDict

result = wikipedia.search("messi ronaldo rivalry")
print(result)

['Messi–Ronaldo rivalry', 'Cristiano Ronaldo', 'Lionel Messi', 'List of career achievements by Cristiano Ronaldo', 'Ronaldo (Brazilian footballer)', 'List of career achievements by Lionel Messi', "2012 FIFA Ballon d'Or", 'New Maradona', "Ballon d'Or", "Ballon d'Or Dream Team"]


In [ ]:
page = wikipedia.page(result[0])
content = page.content

#####**Convert the article into a dictionary**

In [ ]:
content_list = content.split('\n\n\n')

complex_sentences_dict = OrderedDict()
complex_sentences_dict['== Introduction =='] = content_list.pop(0)

for section in content_list:
    section_split = section.split('\n')

    title = section_split.pop(0)
    content = ''
    for para in section_split:
        if para != '':
            content += para
    
    if len(content) > 200:
        complex_sentences_dict[title] = content

In [ ]:
# print the wiki article
for title, content in complex_sentences_dict.items():
    print(title, content, '\n---\n', sep='\n')

== Introduction ==
The Messi–Ronaldo rivalry is an association football rivalry between Argentine forward Lionel Messi and Portuguese forward Cristiano Ronaldo. Having won a combined eleven Ballon d'Or/FIFA Ballon d'Or awards (six for Messi and five for Ronaldo) and ten European Golden Shoe awards (six for Messi and four for Ronaldo), both are widely regarded as two of the greatest players of all time. They are two of the most decorated football players ever, having won a combined 66 trophies (Ronaldo 32, Messi 34) during their senior careers thus far, and have regularly broken the 50-goal barrier in a single season. They are amongst the 51 players in the history of the sport to score over 500 career goals, with both having scored over 700 goals each in their careers for club and country. Ronaldo currently holds the world record for most official goals in a career, while Messi has the superior goals-to-games ratio (0.81, to Ronaldo's 0.73).Journalists and pundits regularly argue the in

---
### Simplify the article

In [ ]:
import nltk.data
nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

sen_length = []
complex_sen_list = []

with open('complex.en', 'w') as fo:
    for title, content in complex_sentences_dict.items():
        content = tokenizer.tokenize(content)
        complex_sen_list = complex_sen_list + content
        sen_length.append(len(content))

        content = '\n'.join(content)
        fo.write(content + '\n')

complex_text = ' '.join(complex_sen_list)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
%run scripts/simplify.py complex.en --model-name muss_en_wikilarge_mined

INFO:fairseq_cli.generate:Namespace(all_gather_list_size=16384, batch_size=None, batch_size_valid=None, beam=5, bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', constraints=None, cpu=False, criterion='cross_entropy', curriculum=0, data='/tmp/tmpv1u5fmdc', data_buffer_size=10, dataset_impl='raw', ddp_backend='c10d', decoding_format=None, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_port=-1, distributed_rank=0, distributed_world_size=1, distributed_wrapper='DDP', diverse_beam_groups=-1, diverse_beam_strength=0.5, diversity_rate=-1.0, empty_cache_freq=0, eval_bleu=False, eval_bleu_args=None, eval_bleu_detok='space', eval_bleu_detok_args=None, eval_bleu_print_samples=False, eval_bleu_remove_bpe=None, eval_tokenized_bleu=False, fast_stat_sync=False, find_unused_parameters=False, fix_batches_to_gpus=False, fixed_validation_seed=None, forc

In [ ]:
import pickle

with open("simple.pkl", "rb") as fo:
    simple_sentences = pickle.load(fo)

simple_text = ' '.join(simple_sentences)

#####**Complex-Simple pairs**

In [ ]:
for simp, comp in zip(simple_sentences, complex_sen_list):
    print(f'Complex== {comp} \nSimple== {simp}\n')

Complex: The Messi–Ronaldo rivalry is an association football rivalry between Argentine forward Lionel Messi and Portuguese forward Cristiano Ronaldo. 
Simple: The Messi-Ronaldo rivalry is a football rivalry between two very good football players: Cristiano Ronaldo and Lionel Messi.

Complex: Having won a combined eleven Ballon d'Or/FIFA Ballon d'Or awards (six for Messi and five for Ronaldo) and ten European Golden Shoe awards (six for Messi and four for Ronaldo), both are widely regarded as two of the greatest players of all time. 
Simple: Many people think that both Messi and Ronaldo are the best players in the world today. Both have won the Ballon d'Or many times, six for Messi and five for Ronaldo, as well as many other awards, such as the European Golden Shoe.

Complex: They are two of the most decorated football players ever, having won a combined 66 trophies (Ronaldo 32, Messi 34) during their senior careers thus far, and have regularly broken the 50-goal barrier in a single se

#####**Simplified Wikipedia Article**

In [ ]:
cur_para, cur_sen = 0, 0
simplified_article = page.original_title + '\n\n'

for lth in sen_length:
    simplified_article += (list(complex_sentences_dict.keys()))[cur_para] + '\n\n'
    for i in range(lth):
        simplified_article += simple_sentences[cur_sen+i] + ' '
    simplified_article += '\n\n'

    cur_para += 1
    cur_sen += lth

print(simplified_article)

Messi–Ronaldo rivalry

== Introduction ==

The Messi-Ronaldo rivalry is a football rivalry between two very good football players: Cristiano Ronaldo and Lionel Messi. Many people think that both Messi and Ronaldo are the best players in the world today. Both have won the Ballon d'Or many times, six for Messi and five for Ronaldo, as well as many other awards, such as the European Golden Shoe. They are two of the most successful football players ever, having won a combined 66 trophies. Messi has won 34 trophies in his senior career, and Ronaldo has won 32 trophies in his senior career. They are the only two players in the history of the sport to score 500 or more goals in a single season. They have both scored over 700 goals in their careers. Ronaldo currently holds the world record for most official goals in a career, while Messi has the superior goals-to-games ratio (0.81, to Ronaldo's 0.73). Many people think that Ronaldo is the best player in the world today, while some people think

#####**Simplified + Summarized Article**

In [ ]:
from transformers import pipeline
summarizer = pipeline('summarization')

INFO:filelock:Lock 140182548512848 acquired on /root/.cache/huggingface/transformers/adac95cf641be69365b3dd7fe00d4114b3c7c77fb0572931db31a92d4995053b.a50597c2c8b540e8d07e03ca4d58bf615a365f134fb10ca988f4f67881789178.lock


Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

INFO:filelock:Lock 140182548512848 released on /root/.cache/huggingface/transformers/adac95cf641be69365b3dd7fe00d4114b3c7c77fb0572931db31a92d4995053b.a50597c2c8b540e8d07e03ca4d58bf615a365f134fb10ca988f4f67881789178.lock
INFO:filelock:Lock 140182561626768 acquired on /root/.cache/huggingface/transformers/b336fa0b874ea92e3e22f07a7e6f8fa9da01221759c33abeb2679d6d98fe7755.585965cf7e82e4536033cd21d76c486af3d6b1c2a34b3a847840d4e7fe9d8844.lock


Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

INFO:filelock:Lock 140182561626768 released on /root/.cache/huggingface/transformers/b336fa0b874ea92e3e22f07a7e6f8fa9da01221759c33abeb2679d6d98fe7755.585965cf7e82e4536033cd21d76c486af3d6b1c2a34b3a847840d4e7fe9d8844.lock
INFO:filelock:Lock 140180001070288 acquired on /root/.cache/huggingface/transformers/f5316f64f9716436994a7ad76a354dc20ecb2dd74eb61d278f103a9c8b80291f.67d01b18f2079bd75eac0b2f2e7235768c7f26bd728e7a855a1c5acae01a91a8.lock


...

INFO:filelock:Lock 140180001070288 released on /root/.cache/huggingface/transformers/f5316f64f9716436994a7ad76a354dc20ecb2dd74eb61d278f103a9c8b80291f.67d01b18f2079bd75eac0b2f2e7235768c7f26bd728e7a855a1c5acae01a91a8.lock
INFO:filelock:Lock 140180001070288 acquired on /root/.cache/huggingface/transformers/9951e68693b9a7c583ae677e9cb53c02715d9bd0311a78706401372653cdea0a.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05.lock


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

INFO:filelock:Lock 140180001070288 released on /root/.cache/huggingface/transformers/9951e68693b9a7c583ae677e9cb53c02715d9bd0311a78706401372653cdea0a.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05.lock
INFO:filelock:Lock 140179999897744 acquired on /root/.cache/huggingface/transformers/7588c8d398d659b230a038240cc023f67b6848117d2999f06ab625af7bfc7ec1.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock


Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

INFO:filelock:Lock 140179999897744 released on /root/.cache/huggingface/transformers/7588c8d398d659b230a038240cc023f67b6848117d2999f06ab625af7bfc7ec1.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock


In [ ]:
char_limit = 2500
summarized_text = ''
for i in range(len(simple_text) // char_limit):
    summarized_text += (summarizer(simple_text[char_limit*i:char_limit*(i+1)], 
                                   max_length=120, min_length=100))[0]['summary_text'] + ' '

print(page.original_title + '\n\n' + summarized_text)

Messi–Ronaldo rivalry

 The Messi-Ronaldo rivalry is a football rivalry between two very good football players . Many people think that both Messi and Ronaldo are the best players in the world today . Messi has won 34 trophies in his senior career, while Ronaldo has won 32 . The two players play against each other at least twice every season in El Clásico . The rivalry has been compared to past global sports rivalries such as the Muhammad Ali-Joe Frazier rivalry in boxing and the Alain Prost-Ayrton Senna rivalry in Formula One .  Messi and Ronaldo finished as runners-up to Kaká in both the Ballon d'Or and FIFA World Player of the Year in 2007 . The two have played against each other at least twice a season since the beginning of the 2009\/10 season, including a two-legged Champions League semifinal in 2011 . Messi has won the award four times in a row (2009 to 2012), while Ronaldo has won it five times . In 2018, Messi made $111 million on the Forbes list of the world's most paid sport

---
### Readability score calculation

In [ ]:
!pip install textstat

     |████████████████████████████████| 102kB 6.7MB/s 
     |████████████████████████████████| 2.0MB 34.0MB/s 


In [ ]:
from textstat import smog_index, flesch_reading_ease, flesch_kincaid_grade, coleman_liau_index 
from textstat import automated_readability_index, dale_chall_readability_score, gunning_fog

print('Readability Index : Complex - Simple\n')

print('Flesch Reading Ease(+) : ', flesch_reading_ease(complex_text), 
      flesch_reading_ease(simple_text))
print('Flesch Kincaid Grade(-) : ', flesch_kincaid_grade(complex_text), flesch_kincaid_grade(simple_text))
print('Coleman Liau Index(-) : ', coleman_liau_index(complex_text), coleman_liau_index(simple_text))
print('Automated Readability Index(-) : ', automated_readability_index(complex_text),
      automated_readability_index(simple_text))
print('Gunning Fog(-) : ', gunning_fog(complex_text), gunning_fog(simple_text))
print('Smog Index(-) : ', smog_index(complex_text), smog_index(simple_text))
print('Dale-Chall Readability Score(-) : ', dale_chall_readability_score(complex_text), 
      dale_chall_readability_score(simple_text))

print('\n---\n\n(+) : higher => better;\n\t90-100 : Very Easy\n\t80-90 : Easy'
      '\n\t70-80 : Fairly Easy\n\t60-70 : Standard\n\t50-60 : Fairly Difficult'
      '\n\t30-50 : Difficult\n\t0-30 : Very Confusing\n')
print('(-) : lower => better;\nThis is a grade formula in that a '
      'score of 9.3 means that a ninth grader would be able to read the document.')

Readability Index : Complex - Simple

Flesch Reading Ease(+) :  42.58 68.3
Flesch Kincaid Grade(-) :  16.5 8.7
Coleman Liau Index(-) :  11.38 9.23
Automated Readability Index(-) :  20.6 10.8
Gunning Fog(-) :  17.15 9.54
Smog Index(-) :  14.4 10.1
Dale-Chall Readability Score(-) :  7.7 6.13

---

(+) : higher => better;
	90-100 : Very Easy
	80-90 : Easy
	70-80 : Fairly Easy
	60-70 : Standard
	50-60 : Fairly Difficult
	30-50 : Difficult
	0-30 : Very Confusing

(-) : lower => better;
This is a grade formula in that a score of 9.3 means that a ninth grader would be able to read the document.


In [ ]:
print('Flesch Reading Ease(+) for the summarised article: ', flesch_reading_ease(summarized_text))

Flesch Reading Ease(+) :  58.42
